In [1]:
from gen_catalyst_design.discrete_space_diffusion import DiffusionModel
from gen_catalyst_design.discrete_space_diffusion.Dataset import get_dataloaders_from_atoms_list
from ase.db import connect
from ase_ml_models.databases import get_atoms_list_from_db
from ase.io import read
import torch.nn.functional as F
import torch
from ase.io import write

In [ ]:
use_absorbing_state = True
mask_classes = True
miller_index = "100"
element_pool = ["Au","Cu","Pd","Rh","Ni","Ga"]
if use_absorbing_state:
    element_pool = ["(X)"] + element_pool

template_db = connect(f"../../databases/templates/{miller_index}/{miller_index}_templates.db")
template_atoms = get_atoms_list_from_db(db_ase=template_db)[0]

diff_model.load_from_checkpoint("AbsorbingStateModel/model_001/checkpoints/epoch=epoch=253-val=val_loss=0.0077.ckpt")
#diff_model.noiser.pre_compute_accum_q_matrices(scheduler=diff_model.scheduler)


TypeError: The classmethod `DiffusionModel.load_from_checkpoint` cannot be called on an instance. Please call it on the class type and make sure the return value is used.

In [10]:
n_samples = 2
batch_size = 40
noised_xs = [diff_model.noiser.sample_from_stationary(num_atoms=len(template_atoms)) for _ in range(n_samples)]

sample_dataset = diff_model.denoiser.get_sample_dataset(
    noised_xs=noised_xs, 
    conditionings=2*torch.ones(size=(len(noised_xs),), dtype=torch.long),
    template_atoms=template_atoms
)

sample_loader = diff_model.denoiser.get_sample_loader(
    dataset=sample_dataset,
    batch_size=batch_size,
    shuffle=False
)

timesteps = torch.arange(diff_model.scheduler.t_init, diff_model.scheduler.t_final+1, 1, dtype=torch.long).flip(dims=(0,))
probs_list = []
for batch in sample_loader:
    denoised_batches = []
    for i, timestep in enumerate(timesteps):
        denoised_batches.append(batch.clone())
        xs_denoised, probs = diff_model.single_denoise_step(
            batch=batch,
            time=timestep,
            guidance_scale=0.8
        )
        probs_list.append(probs)
        batch.x = xs_denoised
        #if i > 3:
        #    break
    denoise_trajes = []
    for sample_idx in range(batch.batch_size):
        denoise_traj = []
        for denoised_batch in denoised_batches:
            graph = denoised_batch.get_example(sample_idx)
            atoms = graph.to_atoms(element_pool)
            denoise_traj.append(atoms)
        denoise_trajes.append(denoise_traj)
    
    for denoise_traj in denoise_trajes:
        write(filename="denoised.traj", images=denoise_traj)
        break
    break


In [4]:
diff_model.scheduler

CosineScheduler()

In [5]:
diff_model.use_x0_reparam

True